# Lab 7
---
DSCI 510 (Hermjakob), Spring 2025, USC

Hello and welcome to Lab 7.

__Guidelines:__

- Please write and submit the programs below by the deadline: Monday, March 10, 2025, at 6:00pm Pacific time.

- You must complete the assignments individually. If you have trouble completing the assignment, please let the teaching assistant (TA) know, during the lab or office hours. He will help and guide you, but he will not write code for you and no one else should :) !!!  

- You have to fill in the code in this notebook and upload it back to Blackboard for submission. Please remember to rename your file as "Lab7_[YOUR FIRSTNAME]_[YOUR LASTNAME].ipynb" (e.g. Lab7_George_Washington.ipynb).

- You may look up resources online like python docs and stackoverflow. You may look up topics, but not the questions themselves.

# Q1 [11 points]
### Medal standing

**Python topics:** requests, csv

To process the csv file, you can use the traditional Python approach (as shown in class), or the csv or pandas libraries (which we will get to after the Spring break).

Load a medal count file from the internet and print out the top teams sorted by performance.

The input medal count file from the web can be assumed to be in csv (comma-separated values) format.<br>
The four input data fields per team are the team's name, and its number of gold, silver, and bronze medals, e.g.
```
Gryffindor,10,6,5
Hufflepuff,1,3,4
Ravenclaw,6,7,10
Slytherin,3,4,1
```

As an output, the function should print a medal standing of the top teams, 
with rank, team name, gold, silver, and bronze count, sorted by the teams' overall medal score 
(2 * number-of-gold-medals + 1.5 * number-of-silver-medals + 1 * number-of-bronze-medals). 
The number of top teams (e.g. 10) should be an input parameter.
Your printout should be in a pretty format (i.e. with columns aligned).

Sample print output for top 3 teams:
```
      Team                G  S  B
  (1) Gryffindor         10  6  5
  (2) Ravenclaw           6  7 10
  (3) Slytherin           3  4  1
```

Before you start, make sure you have the _requests_ library installed:
```
pip install requests
```

In [75]:
import requests
import regex
import sys

def print_medal_standings(csv_url: str, n_teams: int) -> None:
    
    try:
        data = requests.get(csv_url)
    except:
        return sys.stderr.write('URL not found')
    
    team_data = regex.findall(r'\b(\w+?\s*\w+),(\d+),(\d+),(\d+)\b',data.text)
    
    #Initialize dictionary for medal count and list of scores
    team_medals = dict()
    team_scores = []
    
    for team in team_data:
        team_medals[team[0]] = {
            'gold': team[1],
            'silver': team[2],
            'bronze': team[3], 
        }
        team_scores.append((team[0], 2*float(team[1]) + 1.5*float(team[2]) + 1*float(team[3])))
    
    team_scores.sort(key = lambda x: x[1], reverse = True)
    spacer = len(max(team_scores, key = lambda x: len(x[0]))[0]) + 1
    
    scores_table = '     Team' + ' '*(spacer-4) + 'G   S   B'
    for i in range(len(team_scores)):
        team = team_scores[i]
        gold = team_medals[team[0]].get('gold')
        silver = team_medals[team[0]].get('silver')
        bronze = team_medals[team[0]].get('bronze')

        spacer0 = ' '*(3 - len(str(i)))
        spacer1 = ' '*(spacer - len(team_scores[i][0]))
        spacer2 = ' '*(4 - len(gold))
        spacer3 = ' '*(4 - len(silver))
        spacer4 = ' '*(4 - len(bronze))
        scores_table += f'\n({str(i+1)}){spacer0}{team_scores[i][0]}{spacer1}{gold}{spacer2}{silver}{spacer3}{bronze}{spacer4}'

    print(scores_table)


# open test data
print_medal_standings("https://www.dropbox.com/s/dl/x80kix088xspt5v/olympic-medals-2021.csv", 20)

     Team               G   S   B
(1)  United States      39  41  33  
(2)  China              38  32  18  
(3)  Russia             20  28  23  
(4)  Great Britain      22  21  22  
(5)  Japan              27  14  17  
(6)  Australia          17  7   22  
(7)  Italy              10  10  20  
(8)  Germany            10  11  16  
(9)  Netherlands        10  12  14  
(10)  France             10  12  11  
(11) USC                11  5   5   
(12) Canada             7   6   11  
(13) Brazil             7   6   8   
(14) New Zealand        7   6   7   
(15) Hungary            6   7   7   
(16) South Korea        6   4   10  
(17) Spain              3   8   6   
(18) Cuba               7   3   5   
(19) Ukraine            1   6   12  
(20) Poland             4   5   5   
(21) Switzerland        3   4   6   
(22) Czech Republic     4   4   3   
(23) Denmark            3   4   4   
(24) Kenya              4   4   2   
(25) Taiwan             2   4   6   
(26) Turkey             2   2   9   
(27

# Q2: Highlighting with advanced regular expressions 
## [9 regular points + 6 bonus points]
---
It is often helpful to highlight portions of a text. This can be done by adding color or other style elementst.<br>
Below is an example how to color text with an escape sequence:

### Introduction

In [ ]:
# Some style and color escape codes ('\x1B' is the escape character)
reset     = '\x1B[0m'
bold      = '\x1B[1m'
underline = '\x1B[4m'
red       = '\x1B[91m'
green     = '\x1B[92m' 
yellow    = '\x1B[93m' 
blue      = '\x1B[94m' 
print(f"Colors: {red}red{reset}, {green}green{reset}, {blue}blue{reset}, {yellow}yellow{reset}.")
print(f"Styles: {bold}bold{reset}, {underline}underline{reset}.")
print(f"Combination: {red}{bold}{underline}red bold underline{reset}.")

In [ ]:
# Auxiliary function for exploration, testing and debugging. 
import unicodedata as ud
def print_char_unicode_names(s: str) -> None:
    for c in s:
        print(f"{c}  U+{ord(c):04X}  {ud.name(c, '')}")

In [ ]:
print_char_unicode_names("Q: Παρακαλώ;")

### Assignment

Complete the function _highlight6_ below so it highlights 6 different types of "items of interest".

1. Highlight floating point numbers (__already done__, as an example)
2. Highlight sequences of Greek letters, e.g. α or Τροία. Make sure to exclude non-letters such as Greek punctuation.
3. Highlight Latin-script words that contain at least one non-ASCII letter, e.g. Grüße or Österreich.
4. Highlight instances of the generic Chinese classifier 个 (U+4E2A) that are __NOT__ preceded by the following standard Western or Chinese number character \[0123456789一二三四五六七八九十百千万亿兆\]. 
5. __(Bonus)__ Highlight words that contain a Devanagari virama (U+094D) such as मेक्सिको or कैलिफोर्निया. Make sure to include other letter modifiers in the word as well. Background: Most Indian languages use so-called abugida scripts, where consonants have a default vowel (typically 'a'); this default vowel can be overwritten by a vowel sign (a type of letter modifier), or by a virama, a letter modifier that cancels the vowel (without replacing it with another vowel). In the _regex_ library, such letter modifiers are covered in \p{Mark} or \pM for short. See Python regex resource for relevant examples.
6. __(Bonus)__ Highlight words of at least 4 characters, where the first and last characters are the same, and where the second and second last characters are the same, e.g. _reindeer_ or _superstitious_.

All 6 highlight operations should be a __single__ regular expression substitution __line__.<br>
__Note:__ When using regex set operators, please prefix your regular expression with (?V1). See Python regex resource for examples.<br>
__Resource:__ https://uhermjakob.github.io/teaching/python.html<br>
__Grading:__ 3 regular points each for 2.-4. above (1. is already done); 3 bonus points each for 5.-6. above. 

In [ ]:
import regex

def highlight6(text: str) -> str:
    # Instead of directly using highlight escape sequences, we use temporary placeholders
    # in order not to interfere with any highlighting operations.
    text = regex.sub('[\uE000-\uE001]', '', text)  # delete any of the following two private use area characters
    s = '\uE000'  # start highlight placeholder character (private use area)
    e = '\uE001'  # end highlight placeholder character (private use area)

    # Highlight floats (e.g. 3.14)
    text = regex.sub(r'(-?\d+\.\d+)', fr'{s}\1{e}', text)  # Provided as an example.
    
    # Highlight sequences of Greek letters
    # text = regex.sub(ADD_REGEX_HERE)

    # Highlight words with non-ASCII Latin letters
    # text = regex.sub(ADD_REGEX_HERE)

    # Highlight Chinese classifier 个 if NOT preceded by standard number character.
    # text = regex.sub(ADD_REGEX_HERE)

    # Highlight words with a Devanagari sign virama
    # text = regex.sub(ADD_REGEX_HERE)

    # Highlight words where first and last, second and second-last letters are the same.
    # text = regex.sub(ADD_REGEX_HERE)
      
    # Replace private-use-area start/end highlight placeholder characters with actual highlight escape sequences
    text = regex.sub(s, '\x1B[1m\x1B[91m', text)  # escape sequence for: bold red
    text = regex.sub(e, '\x1B[0m', text)          # escape sequence for: reset
    
    return text

In [ ]:
open_tests = ['The temperature was 20.5°C (68.9°F).']

for test in open_tests:
    print(highlight6(test))